In [13]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np

In [14]:
dfvino = pd.read_csv(r"c:\Users\usuari\Desktop\Proyecto_final\data\dataVino2023.csv")

#### Visión general de los datos

In [15]:
#Muestra la cantidad de filas y columnas del archivo
dfvino.shape    

(8364, 7)

In [16]:
# Muestra el tipo de variable de cada columna
dfvino.dtypes

Continent         object
Region/Country    object
Product           object
Variable          object
Year               int64
Unit              object
Quantity          object
dtype: object

### Correcciones de datos

In [17]:
# para corregir el espacio de separacion de miles de la columna 'Quantity'
dfvino['Quantity'] = dfvino['Quantity'].str.replace(' ', '', regex=False).astype(float)

In [18]:
#Convierte la columna Quantity a tipo de dato numérico
dfvino['Quantity'] = pd.to_numeric(dfvino['Quantity'], errors='coerce')  

### Visualización de las cuatro "regiones de China" a unir

In [19]:
ubicaciones = ['China, continental', 'China, RAE de Hong Kong', 'China, RAE de Macao', 'China, Taiwán provincia de']  # Especifica tus 4 ubicaciones

df_para_visualizar = dfvino[dfvino["Region/Country"].isin(ubicaciones)][["Region/Country", "Year", "Variable", "Quantity"]]

print(df_para_visualizar)

                  Region/Country  Year       Variable  Quantity
1631          China, continental  2013        Consumo   18714.0
1632          China, continental  2013  Exportaciones      19.0
1633          China, continental  2013  Importaciones    3766.0
1634          China, continental  2013     Producción   13693.0
1635          China, continental  2014        Consumo   17369.0
...                          ...   ...            ...       ...
1800  China, Taiwán provincia de  2022     Producción       0.0
1801  China, Taiwán provincia de  2023        Consumo     212.0
1802  China, Taiwán provincia de  2023  Exportaciones       0.0
1803  China, Taiwán provincia de  2023  Importaciones     238.0
1804  China, Taiwán provincia de  2023     Producción       0.0

[174 rows x 4 columns]


In [20]:
df_sumatorio = df_para_visualizar[df_para_visualizar["Region/Country"].isin(ubicaciones)].groupby("Region/Country")["Quantity"].sum().reset_index()
print(df_sumatorio)

               Region/Country  Quantity
0     China, RAE de Hong Kong   10643.0
1         China, RAE de Macao    1189.0
2  China, Taiwán provincia de    4512.0
3          China, continental  321859.0


In [21]:
suma_general = dfvino[dfvino["Region/Country"].isin(ubicaciones)]["Quantity"].sum()
print(suma_general)

338203.0


### Unificación de 'China, continental', 'China, RAE de Hong Kong', 'China, RAE de Macao', 'China, Taiwán provincia de' en una sola región, 'China' 

In [22]:
# Unificiación de las versiones de China
# Filtrar las filas de las 4 versiones de China
paises_china = ['China, continental', 'China, RAE de Hong Kong', 'China, RAE de Macao', 'China, Taiwán provincia de']
df_china = dfvino[dfvino['Region/Country'].isin(paises_china)]

# Agrupar las filas de las versiones de China por 'Year' y 'Variable' y sumar la columna 'Quantity'
df_china_agrupado = df_china.groupby(['Continent', 'Product','Year', 'Variable','Unit'], as_index=False)['Quantity'].sum()

# Reemplazar las filas de las versiones de China originales con las filas unificadas de China
# Asegurándonos de que para cada combinación de Year y Variable haya 1 sola fila de China

# Cambiar el nombre de la columna 'Region/Country' a 'China'
df_china_agrupado['Region/Country'] = 'China'

# Eliminar las filas originales de las 4 versiones de China en dfvino
dfvino = dfvino[~dfvino['Region/Country'].isin(paises_china)]

# Concatenar las filas agrupadas de China al DataFrame original
dfvino_union = pd.concat([dfvino, df_china_agrupado], ignore_index=True)


### Comprobación de que la unión se ha hecho correctamente y que no se han alterado datos

In [23]:
dfvino_union.shape

(8234, 7)

In [24]:
dfvino_union['Region/Country'].unique()

array(['Afganistán', 'Albania', 'Alemania', 'American Samoa', 'Andorra',
       'Angola', 'Antigua y Barbuda', 'Antillas Neerlandesas',
       'Arabia Saudí', 'Argelia', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaiyán', 'Bahamas', 'Bangladés',
       'Barbados', 'Baréin', 'Bélgica', 'Belgium and Luxemburg', 'Belice',
       'Benín', 'Bermuda', 'Bielorrusia', 'Bolivia',
       'Bosnia y Herzegovina', 'Botsuana', 'Brasil', 'Brunéi', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Bután', 'Cabo Verde', 'Camboya',
       'Camerún', 'Canadá', 'Cayman Islands', 'Chad', 'Chile', 'Chipre',
       'Colombia', 'Comoras', 'Congo', 'Cook Islands', 'Corea del Norte',
       'Corea del Sur', 'Costa de Marfil', 'Costa Rica', 'Croacia',
       'Cuba', 'Dinamarca', 'Dominica', 'Ecuador', 'Egipto',
       'El Salvador', 'Emiratos Árabes Unidos', 'Eritrea', 'Eslovaquia',
       'Eslovenia', 'España', 'Estados Unidos', 'Estonia', 'Esuatini',
       'Etiopía', 'Faroe Islands', 'F

In [25]:
# Comprobación de que la unificación de China se ha realizado correctamente mediante suma de la columna 'Quantity'
dfvino_union[dfvino_union['Region/Country'] == 'China']['Quantity'].sum()

np.float64(338203.0)

##### La suma coincide con la anterior

In [26]:
dfvino_union.head()

,Continent,Region/Country,Product,Variable,Year,Unit,Quantity
0,Asia,Afganistán,Vino,Consumo,2013,1000 hl,1.0
1,Asia,Afganistán,Vino,Importaciones,2013,1000 hl,0.0
2,Asia,Afganistán,Vino,Producción,2013,1000 hl,0.0
3,Asia,Afganistán,Vino,Consumo,2014,1000 hl,1.0
4,Asia,Afganistán,Vino,Importaciones,2014,1000 hl,1.0


In [27]:
#Produccion
df_produccion = dfvino_union[dfvino_union['Variable'] == 'Producción']

#Agrupacion de país y sumar cantidades
produccion_pais = (df_produccion.groupby('Region/Country')['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False))

print(produccion_pais.head(10))

     Region/Country  Quantity
99           Italia  531422.0
72          Francia  487993.0
63           España  414354.0
64   Estados Unidos  267191.0
13        Australia  136015.0
10        Argentina  135894.0
41            Chile  127302.0
187       Sudáfrica  115095.0
42            China  102869.0
2          Alemania   95777.0


In [28]:
#Exportaciones
df_exportaciones = dfvino_union[dfvino_union['Variable'] == 'Exportaciones']

#Agrupacion de país y sumar cantidades
exportaciones_pais = (df_exportaciones.groupby('Region/Country')['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False))

print(exportaciones_pais.head(10))

     Region/Country  Quantity
49           España  240629.0
73           Italia  229406.0
56          Francia  154597.0
31            Chile   93375.0
8         Australia   79586.0
135       Sudáfrica   51498.0
1          Alemania   40679.0
50   Estados Unidos   38401.0
113        Portugal   33241.0
5         Argentina   31079.0


In [29]:
#Importaciones
df_importaciones = dfvino_union[dfvino_union['Variable'] == 'Importaciones']

#Agrupacion de país y sumar cantidades
importaciones_pais = (df_importaciones.groupby('Region/Country')['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False))

print(importaciones_pais.head(10))

     Region/Country  Quantity
2          Alemania  162170.0
152     Reino Unido  147404.0
61   Estados Unidos  132814.0
69          Francia   73224.0
41            China   62652.0
147    Países Bajos   47145.0
37           Canadá   45057.0
160           Rusia   44280.0
33          Bélgica   34825.0
98            Japón   29007.0


In [30]:
#Consumo
df_consumo = dfvino_union[dfvino_union['Variable'] == 'Consumo']

#Agrupacion de país y sumar cantidades
consumo_pais = (df_consumo.groupby('Region/Country')['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False))

print(consumo_pais.head(10))

     Region/Country  Quantity
56   Estados Unidos  358653.0
64          Francia  286953.0
90           Italia  244152.0
2          Alemania  218284.0
37            China  170227.0
142     Reino Unido  143166.0
55           España  110156.0
9         Argentina   99668.0
150           Rusia   98442.0
11        Australia   60267.0


In [31]:
#Comprobacion de suma de importacion, consumo y produccion de Italia, Francia, China continental, Alemania, Estados Unidos y Reino Unido
#Creo una variable con los paises de interes
paises_interes = ['Italia', 'Francia', 'China', 'Alemania', 'Estados Unidos', 'Reino Unido', 'España']

#Creo un dataframe con los paises de interes y los valores de Produccion, Importaciones y Consumo
df_paises_interes = dfvino_union[(dfvino_union['Region/Country'].isin(paises_interes)) & (dfvino_union['Variable'].isin(['Producción', 'Importaciones', 'Consumo']))]

#Agrupacion de país y sumar cantidades
df_total_pais_variables = df_paises_interes.groupby('Region/Country')['Quantity'].sum().reset_index().sort_values(by='Quantity', ascending=False)
print(df_total_pais_variables)

   Region/Country  Quantity
4         Francia  848170.0
5          Italia  799881.0
3  Estados Unidos  758658.0
2          España  532543.0
0        Alemania  476231.0
1           China  335748.0
6     Reino Unido  291330.0


### Creación del nuevo csv para su posterior tratamiento

In [14]:
#dfvino_unido = dfvino.copy()
#dfvino_unido.to_csv('dfvino_unido.csv', index=False)